<a href="https://colab.research.google.com/github/UmaNagirireddi/ML_7/blob/main/ML7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.9 MB/s eta 0:00:00
time: 240 µs (started: 2023-12-12 23:52:36 +00:00)


In [ ]:
import os, torch, torch.nn as nn, torch.optim as optim, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)
np.random.seed(123)


time: 6.57 s (started: 2023-12-12 23:53:08 +00:00)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

time: 57.2 ms (started: 2023-12-12 23:53:36 +00:00)


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

time: 487 µs (started: 2023-12-12 23:54:19 +00:00)


In [ ]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean, std = imgs.view(3, -1).mean(dim=1), imgs.view(3, -1).std(dim=1)
mean, std

100%|██████████| 170498071/170498071 [00:13<00:00, 12813771.58it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
time: 27.7 s (started: 2023-12-12 23:56:37 +00:00)


In [ ]:
cifar10 = datasets.CIFAR10('./data', train=True, download=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]))


time: 502 ms (started: 2023-12-12 23:58:19 +00:00)


In [ ]:
cifar10_val = datasets.CIFAR10('./data', train=False, download=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]))


time: 376 ms (started: 2023-12-12 23:59:33 +00:00)


In [ ]:
train_loader, test_loader = (DataLoader(ds, batch_size=32, shuffle=s, num_workers=2) for ds, s in ((cifar10, True), (cifar10_val, False)))
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']


time: 1.05 ms (started: 2023-12-13 00:00:57 +00:00)


In [ ]:
import datetime
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 2 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))


time: 718 µs (started: 2023-12-13 00:01:53 +00:00)


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # <1>
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # <2>
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x


time: 647 µs (started: 2023-12-13 00:02:09 +00:00)


In [ ]:
import torch.nn.functional as F
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out


time: 828 µs (started: 2023-12-13 00:03:44 +00:00)


In [ ]:
Model_n = NetResDeep(n_chans1=32, n_blocks=10).to(device=device)
optimizer_n = optim.SGD(Model_n.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()
training_loop(
    n_epochs = 300,
    optimizer = optimizer_n,
    model = Model_n,
    loss_fn = loss_fn,
    train_loader = train_loader,
)


2023-12-13 00:06:01.542054 Epoch 1, Training loss 1.672733060457885
2023-12-13 00:06:18.609902 Epoch 2, Training loss 1.3218849350913395
2023-12-13 00:06:52.164532 Epoch 4, Training loss 1.0942304098522213
2023-12-13 00:07:25.282308 Epoch 6, Training loss 0.9790652731596775
2023-12-13 00:07:59.566404 Epoch 8, Training loss 0.8980753872345749
2023-12-13 00:08:33.005611 Epoch 10, Training loss 0.8390941984250769
2023-12-13 00:09:06.558077 Epoch 12, Training loss 0.7891363979186755
2023-12-13 00:09:40.708258 Epoch 14, Training loss 0.7482401559883711
2023-12-13 00:10:14.187360 Epoch 16, Training loss 0.7153825359853963
2023-12-13 00:10:47.388864 Epoch 18, Training loss 0.6806132789899048
2023-12-13 00:11:22.498649 Epoch 20, Training loss 0.6513970437449518
2023-12-13 00:11:56.814788 Epoch 22, Training loss 0.6244742403370558
2023-12-13 00:12:30.262738 Epoch 24, Training loss 0.5968108157450316
2023-12-13 00:13:04.614420 Epoch 26, Training loss 0.5738908010210201
2023-12-13 00:13:38.807137

In [ ]:
trainLoader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
valLoader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
def validate(model, trainLoader, valLoader):
    accdict = {}
    predictions = []
    exp_labels = []

    for name, loader in [("train", train_loader), ("val", valLoader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

                predictions.extend(predicted.cpu().numpy())
                exp_labels.extend(labels.cpu().numpy())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict, predictions, exp_labels


time: 1.24 ms (started: 2023-12-13 01:38:47 +00:00)


In [ ]:
accuracy_n, predictions_n, expectedLabels = validate(Model_n, trainLoader, valLoader)
precision_n, recall_n, cnfMatrix = precision_score(predictions_n, expectedLabels, average='macro'), recall_score(predictions_n, expectedLabels, average='macro'), confusion_matrix(predictions_n, expectedLabels)
print(cnfMatrix)

Accuracy train: 1.00
Accuracy val: 0.66
[[5638   30   73   27   25   17    3   15  100   41]
 [  29 5786   10   21    8    5    6   10   52   98]
 [  98   10 5521   71  110   64   65   43   25   15]
 [  28    9   73 5444   69  166   64   39   21   16]
 [  22    4   90   71 5504   41   48   49   12    9]
 [  16    7   75  191   69 5559   36   71   13   10]
 [  15   11   74   76   67   36 5743    9   10   11]
 [  23    4   65   67  127   89   19 5745   10   19]
 [  77   33    8   16   15   12    8    3 5722   27]
 [  54  106   11   16    6   11    8   16   35 5754]]
time: 11.5 s (started: 2023-12-13 01:38:48 +00:00)


In [ ]:
print(classification_report(predictions_n, expectedLabels, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.94      0.94      0.94      5969
  automobile       0.96      0.96      0.96      6025
        bird       0.92      0.92      0.92      6022
         cat       0.91      0.92      0.91      5929
        deer       0.92      0.94      0.93      5850
         dog       0.93      0.92      0.92      6047
        frog       0.96      0.95      0.95      6052
       horse       0.96      0.93      0.94      6168
        ship       0.95      0.97      0.96      5921
       truck       0.96      0.96      0.96      6017

    accuracy                           0.94     60000
   macro avg       0.94      0.94      0.94     60000
weighted avg       0.94      0.94      0.94     60000

time: 143 ms (started: 2023-12-13 01:39:00 +00:00)


In [ ]:
class NetWidth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out



time: 782 µs (started: 2023-12-13 01:39:00 +00:00)


In [ ]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn,
                        train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                          for p in model.parameters())  # <1>
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))


time: 679 µs (started: 2023-12-13 01:39:00 +00:00)


In [ ]:
modelWd = NetWidth(n_chans1=32).to(device=device)
training_loop_l2reg(300, optim.SGD(modelWd.parameters(), lr=1e-2), modelWd, nn.CrossEntropyLoss(), trainLoader)


2023-12-13 01:39:15.408756 Epoch 1, Training loss 1.9960977820789112
2023-12-13 01:41:30.030288 Epoch 10, Training loss 1.1806000449011087
2023-12-13 01:43:59.481680 Epoch 20, Training loss 0.9877848781435691
2023-12-13 01:46:29.565967 Epoch 30, Training loss 0.9088765910976683
2023-12-13 01:48:59.064253 Epoch 40, Training loss 0.8647717310644477
2023-12-13 01:51:27.926125 Epoch 50, Training loss 0.8352194234080936
2023-12-13 01:53:56.840882 Epoch 60, Training loss 0.8138626804742057
2023-12-13 01:56:26.624072 Epoch 70, Training loss 0.7976617735365162
2023-12-13 01:58:56.414703 Epoch 80, Training loss 0.7849743370052493
2023-12-13 02:01:26.380860 Epoch 90, Training loss 0.7748633558335511
2023-12-13 02:03:56.590332 Epoch 100, Training loss 0.7665573676376392
2023-12-13 02:06:26.407034 Epoch 110, Training loss 0.759614470517239
2023-12-13 02:08:55.635963 Epoch 120, Training loss 0.7537946329854638
2023-12-13 02:11:25.725706 Epoch 130, Training loss 0.7488175724321009
2023-12-13 02:13:5

In [ ]:
accuracyWd, predictionsWd, expectedLabelsWd = validate(modelWd, trainLoader, valLoader)
precisionWd, recallWd = (precision_score(predictionsWd, expectedLabelsWd, average='macro'),
                         recall_score(predictionsWd, expectedLabelsWd, average='macro'))
cnfMatrixWd = confusion_matrix(predictionsWd, expectedLabelsWd)
print(cnfMatrix)

Accuracy train: 0.87
Accuracy val: 0.69
[[5638   30   73   27   25   17    3   15  100   41]
 [  29 5786   10   21    8    5    6   10   52   98]
 [  98   10 5521   71  110   64   65   43   25   15]
 [  28    9   73 5444   69  166   64   39   21   16]
 [  22    4   90   71 5504   41   48   49   12    9]
 [  16    7   75  191   69 5559   36   71   13   10]
 [  15   11   74   76   67   36 5743    9   10   11]
 [  23    4   65   67  127   89   19 5745   10   19]
 [  77   33    8   16   15   12    8    3 5722   27]
 [  54  106   11   16    6   11    8   16   35 5754]]
time: 10.4 s (started: 2023-12-13 02:53:58 +00:00)


In [ ]:
print(classification_report(predictionsWd, expectedLabelsWd, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.83      0.86      0.84      5778
  automobile       0.92      0.92      0.92      5994
        bird       0.77      0.76      0.76      6058
         cat       0.75      0.71      0.73      6404
        deer       0.79      0.81      0.80      5845
         dog       0.80      0.76      0.78      6306
        frog       0.88      0.86      0.87      6166
       horse       0.86      0.86      0.86      5996
        ship       0.87      0.92      0.89      5674
       truck       0.89      0.92      0.90      5779

    accuracy                           0.84     60000
   macro avg       0.84      0.84      0.84     60000
weighted avg       0.84      0.84      0.84     60000

time: 144 ms (started: 2023-12-13 02:54:08 +00:00)


In [ ]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

time: 806 µs (started: 2023-12-13 02:54:08 +00:00)


In [ ]:
modelDropout = NetDropout(n_chans1=32).to(device)
training_loop(100, optim.SGD(modelDropout.parameters(), lr=1e-2), modelDropout, nn.CrossEntropyLoss(), trainLoader)


2023-12-13 02:54:28.806272 Epoch 1, Training loss 2.010632708706819
2023-12-13 02:54:42.951110 Epoch 2, Training loss 1.7617251638256375
2023-12-13 02:55:10.934620 Epoch 4, Training loss 1.5603083641937627
2023-12-13 02:55:38.843249 Epoch 6, Training loss 1.4585976015271434
2023-12-13 02:56:06.865842 Epoch 8, Training loss 1.3876442375695308
2023-12-13 02:56:34.822441 Epoch 10, Training loss 1.3370900048929102
2023-12-13 02:57:02.818491 Epoch 12, Training loss 1.2908255769621075
2023-12-13 02:57:30.767301 Epoch 14, Training loss 1.260003665066741
2023-12-13 02:57:58.780888 Epoch 16, Training loss 1.2269336489003029
2023-12-13 02:58:26.729799 Epoch 18, Training loss 1.1982291987942308
2023-12-13 02:58:54.620025 Epoch 20, Training loss 1.175606329270336
2023-12-13 02:59:22.839701 Epoch 22, Training loss 1.1502697816895098
2023-12-13 02:59:51.016604 Epoch 24, Training loss 1.1386213134926604
2023-12-13 03:00:19.286723 Epoch 26, Training loss 1.1216915629403976
2023-12-13 03:00:47.198654 E

In [ ]:
accuracyDropout, predictionsDropout, expectedLabelsDropout = validate(modelDropout, trainLoader, valLoader)

Accuracy train: 0.67
Accuracy val: 0.61
time: 9.98 s (started: 2023-12-13 03:17:39 +00:00)


In [ ]:
precisionDropout = precision_score(predictionsDropout, expectedLabelsDropout, average='macro')
recallDropout = recall_score(predictionsDropout, expectedLabelsDropout, average='macro')
cnfMatrixDropout = confusion_matrix(predictionsDropout, expectedLabelsDropout)
print(cnfMatrixDropout)

[[4295  155  424  125  189   54   52   75  600  219]
 [ 153 4601   41   67   38   44   46   43  273  595]
 [ 317   58 3184  390  522  426  228  221  100   55]
 [ 179   90  387 2938  400 1159  402  355  131  131]
 [ 108   44  694  417 3575  423  328  459   54   45]
 [  66   32  334  887  214 3047  118  346   31   43]
 [  61   91  540  710  541  344 4681  123   64  100]
 [ 107   54  224  256  406  378   56 4176   48  143]
 [ 431  198  103  100   59   54   40   49 4436  156]
 [ 283  677   69  110   56   71   49  153  263 4513]]
time: 166 ms (started: 2023-12-13 03:17:49 +00:00)


In [ ]:
print(classification_report(predictionsDropout, expectedLabelsDropout, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.72      0.69      0.70      6188
  automobile       0.77      0.78      0.77      5901
        bird       0.53      0.58      0.55      5501
         cat       0.49      0.48      0.48      6172
        deer       0.60      0.58      0.59      6147
         dog       0.51      0.60      0.55      5118
        frog       0.78      0.65      0.71      7255
       horse       0.70      0.71      0.70      5848
        ship       0.74      0.79      0.76      5626
       truck       0.75      0.72      0.74      6244

    accuracy                           0.66     60000
   macro avg       0.66      0.66      0.66     60000
weighted avg       0.66      0.66      0.66     60000

time: 145 ms (started: 2023-12-13 03:17:49 +00:00)


In [ ]:
class NetBatchNorm(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_batchnorm = nn.BatchNorm2d(num_features=n_chans1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_batchnorm = nn.BatchNorm2d(num_features=n_chans1 // 2)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.conv1_batchnorm(self.conv1(x))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = self.conv2_batchnorm(self.conv2(out))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

time: 852 µs (started: 2023-12-13 03:17:49 +00:00)


In [ ]:
modelBatchNorm = NetBatchNorm(n_chans1=32).to(device=device)
optimizerBatchNorm = optim.SGD(modelBatchNorm.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizerBatchNorm,
    model = modelBatchNorm,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2023-12-13 03:17:57.635948 Epoch 1, Training loss 1.6479532561924544
2023-12-13 03:18:05.239507 Epoch 2, Training loss 1.321204065018103
2023-12-13 03:18:20.174032 Epoch 4, Training loss 1.0985896263226285
2023-12-13 03:18:35.154568 Epoch 6, Training loss 0.9815801281770375
2023-12-13 03:18:50.328404 Epoch 8, Training loss 0.9080408466640223
2023-12-13 03:19:05.068961 Epoch 10, Training loss 0.8561229745470708
2023-12-13 03:19:20.264188 Epoch 12, Training loss 0.8144746814221049
2023-12-13 03:19:35.246354 Epoch 14, Training loss 0.7799158152371588
2023-12-13 03:19:50.625667 Epoch 16, Training loss 0.7530139590484243
2023-12-13 03:20:05.781374 Epoch 18, Training loss 0.7254330565818059
2023-12-13 03:20:21.327853 Epoch 20, Training loss 0.701974105163789
2023-12-13 03:20:36.509911 Epoch 22, Training loss 0.6819875512417508
2023-12-13 03:20:51.586974 Epoch 24, Training loss 0.662671828529275
2023-12-13 03:21:06.589860 Epoch 26, Training loss 0.6426331825883284
2023-12-13 03:21:21.965637 E

In [ ]:
accuracyBatchNorm, predictionsBatchNorm, expectedLabelsBatchNorm = validate(modelDropout, trainLoader, valLoader)

Accuracy train: 0.67
Accuracy val: 0.60
time: 10 s (started: 2023-12-13 03:30:28 +00:00)


In [ ]:
precisionBatchNorm = precision_score(predictionsBatchNorm, expectedLabelsBatchNorm, average='macro')
recallBatchNorm = recall_score(predictionsBatchNorm, expectedLabelsBatchNorm, average='macro')
cnfMatrixBatchNorm = confusion_matrix(predictionsBatchNorm, expectedLabelsBatchNorm)
print(cnfMatrixBatchNorm)

[[4272  156  426  124  212   48   49   81  547  208]
 [ 158 4580   54   58   34   49   48   48  272  566]
 [ 314   51 3173  418  570  436  271  241  104   55]
 [ 177   80  398 2996  408 1138  390  313  127  168]
 [ 120   48  667  398 3568  431  320  502   39   37]
 [  53   37  344  876  236 3096   98  346   40   39]
 [  55   99  533  683  476  351 4687  128   73  103]
 [ 100   59  246  255  401  344   48 4176   43  148]
 [ 465  203  104   84   49   37   42   39 4480  164]
 [ 286  687   55  108   46   70   47  126  275 4512]]
time: 165 ms (started: 2023-12-13 03:30:38 +00:00)


In [ ]:
print(classification_report(predictionsDropout, expectedLabelsDropout, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.72      0.69      0.70      6188
  automobile       0.77      0.78      0.77      5901
        bird       0.53      0.58      0.55      5501
         cat       0.49      0.48      0.48      6172
        deer       0.60      0.58      0.59      6147
         dog       0.51      0.60      0.55      5118
        frog       0.78      0.65      0.71      7255
       horse       0.70      0.71      0.70      5848
        ship       0.74      0.79      0.76      5626
       truck       0.75      0.72      0.74      6244

    accuracy                           0.66     60000
   macro avg       0.66      0.66      0.66     60000
weighted avg       0.66      0.66      0.66     60000

time: 145 ms (started: 2023-12-13 03:30:38 +00:00)


time: 153 ms (started: 2023-12-13 03:30:38 +00:00)
